In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# import CRUD python module and AnimalShelter class
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Access for MongoDB authentication and creation of class object
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

# load database info into pandas dataframe 
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# insert Grazioso Salvare logo image
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# dash app layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Matthew Neale'))),
    html.Hr(),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Div('Data Filter: '),
    
    # dropdown selection for filter on datatable
    html.Div([
        dcc.Dropdown(
        
            id='filter-type',
            options=[
                {'label': 'No Filter', 'value': None},
                {'label': 'Water Rescue', 'value': 1},
                {'label': 'Mountain or Wilderness Rescue', 'value': 2},
                {'label': 'Disaster or Individual Tracking', 'value': 3}
            ],
            value = None
        ),
        
    ]),        
    html.Hr(),
    
    # datatable to display database information
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10, 
        
    ),
    html.Br(),
    html.Hr(),
    
    # dropdown filter options for the pie chart
    html.Div(className = 'row',
            style = {'width':'30%'},
            children = [
                html.Div('Pie Chart Filter: '),
                html.Div(
                dcc.Dropdown(
                    id = 'dd-pie-chart',
                options = [
                    {'label':'Breed','value':1},
                    {'label':'Age', 'value':2}
                ],
                value = 1
                ),
                ),
            ]),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# callback for updating the dashboard datatable based on filter selected
@app.callback(
    [Output('datatable-id','data'),
    Output('datatable-id','columns')],
    [Input('filter-type', 'value')])
def update_dashboard(value):
    # No Filter selected
    if value is None:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    # Water Rescue selected
    elif value is 1:
        df = pd.DataFrame.from_records(shelter.read({'$and':[
            {'$or':[ {'breed':'Newfoundland'},
                     {'breed':'Labrador Retriever Mix'},
                     {'breed':'Chesapeake Bay Retriever'}]},
            
            {'sex_upon_outcome':'Intact Female'},
            
            {'$and':[ {'age_upon_outcome_in_weeks': {'$gte': 26}},
                      {'age_upon_outcome_in_weeks': {'$lte': 156}}
                   ]}
        ]}))
        
    # Mountain and Wilderness Rescue selected
    elif value is 2:
        df = pd.DataFrame.from_records(shelter.read({'$and':[
            {'$or':[ {'breed':'German Shepherd'},
                     {'breed':'Alaskan Malamute'},
                     {'breed':'Old English Sheepdog'},
                     {'breed':'Siberian Husky'},
                     {'breed':'Rottweiler'}]},
            
            {'sex_upon_outcome':'Intact Male'},
            
            {'$and':[ {'age_upon_outcome_in_weeks': {'$gte': 26}},
                      {'age_upon_outcome_in_weeks': {'$lte': 156}}
                   ]}
        ]}))
        
    # Disaster and Individual Tracking selected
    elif value is 3:
        df = pd.DataFrame.from_records(shelter.read({'$and':[
            {'$or':[ {'breed':'Doberman Pinscher'},
                     {'breed':'German Shepherd'},
                     {'breed':'Golden Retriever'},
                     {'breed':'Bloodhound'},
                     {'breed':'Rottweiler'}]},
            
            {'sex_upon_outcome':'Intact Male'},
            
            {'$and':[ {'age_upon_outcome_in_weeks': {'$gte': 20}},
                      {'age_upon_outcome_in_weeks': {'$lte': 300}}
                   ]}
        ]}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)



# callback for highlighting selected row
### FIXME should highlight only selected row, not all of them
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]

# callback for pie chart based on selected filter options
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('dd-pie-chart', 'value')])
def update_graphs(viewData, value):
    df = pd.DataFrame.from_dict(viewData)
    
    # Breed filter selected
    if value is 1:
        fig = px.pie(df, names = 'breed')
        
    # Age filter selected
    elif value is 2:
        fig = px.pie(df, names = 'age_upon_outcome')
        
    return [
        dcc.Graph(
            figure = fig)
    ]

# callback for displaying geolocation map with marker pointing to first animal on the currently
# displayed list
### FIXME should point to animal in currently selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
        # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.H3("Matthew Neale Wuz Here"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]



app